In [1]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
# python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cosmi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
# import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models # !!! name changed from gensim to gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
# Read CSV file containing preprocessed parleys as pandas dataframe
import pandas as pd

parler_df = pd.read_csv('C:\\Users\\cosmi\\Desktop\\ANDREEA\\bachelors-thesis\\parler_df_dates_jan.csv')
print(parler_df)

                                                     body createdAtformatted
0                                              biden pedo           07-01-21
1       qanonsense know shock not need government look...           02-01-21
2                        sjwhunter fucked should have cap           08-01-21
3                         presidentbiden life bring pussy           01-01-21
4                                                snailman           05-01-21
...                                                   ...                ...
110565                         mustwatchvideo second know           02-01-21
110566                                    petition recall           01-01-21
110567                             incompetent idiot fool           05-01-21
110568                           mob bos trafficking cult           02-01-21
110569  did not believe watch video president obama te...           07-01-21

[110570 rows x 2 columns]


In [8]:
# Convert pandas dataframe to list
data = parler_df.body.values.tolist()
data

['biden pedo',
 'qanonsense know shock not need government looking unlike fattso lol',
 'sjwhunter fucked should have cap',
 'presidentbiden life bring pussy',
 'snailman',
 'fucktrumptohell pussy cannot nothing that care fuck either way like fucking lazy that all',
 'edanielc traitor',
 'encepthor you are boring you spine',
 'rickyclabough smart safe',
 'marthell that told who dementia joe msm deep state bureaucrat that not truth though looking burisma corruption stopped dementia think tax dollar blackmail another country that investigating kid business dealing democrat impeach trump trying get bottom country politician corrupt used every mean possible get rid guy revealing corruption biden whole family dirty',
 'williamalmeida keep cry magat enjoy new president disgrace are',
 'marklake ehh have done pretty well life thanks concern',
 'sjwhunter see female racist trump supporter get killed yesterday time almost fuckboy',
 'football rolling stone give shelter live official promo',
 's

In [9]:
# 8. Tokenize words and Clean-up text

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  

data_words = list(sent_to_words(data))

print(data_words[:1])

NameError: name 'gensim' is not defined

In [ ]:
# 9. Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# 10. Remove Stopwords, Make Bigrams and Lemmatize

# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en 
# !!! As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full pipeline package name 'en_core_web_sm' instead.
# nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# 11. Create the Dictionary and Corpus needed for Topic Modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

id2word[0]

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
# 12. Building the Topic Model

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# 13. View the topics in LDA model

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# 14. Compute Model Perplexity and Coherence Score

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# 15. Visualize the topics-keywords

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis